Physionet Challenge

- https://physionet.org/challenge/2019/

Reduce unncessary lab tests with AI  
https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5694620/

In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM
from keras.models import Sequential
from keras.layers import Dense
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

Using TensorFlow backend.


In [2]:
pat_sample_df = pd.read_csv('../../../datasets/training_setA/training/p000001.psv', sep='|')
pat_sample_df.describe()
pat_sample_df.columns
# 40 time-dependent variables + sepsis_label (SepsisLabel is 1 if t≥tsepsis−6 and 0 if t<tsepsis−6. For non-sepsis patients, SepsisLabel is 0.)

Index(['HR', 'O2Sat', 'Temp', 'SBP', 'MAP', 'DBP', 'Resp', 'EtCO2', 'BaseExcess', 'HCO3', 'FiO2', 'pH', 'PaCO2', 'SaO2', 'AST', 'BUN', 'Alkalinephos', 'Calcium', 'Chloride', 'Creatinine', 'Bilirubin_direct', 'Glucose', 'Lactate', 'Magnesium', 'Phosphate', 'Potassium', 'Bilirubin_total', 'TroponinI', 'Hct', 'Hgb', 'PTT', 'WBC', 'Fibrinogen', 'Platelets', 'Age', 'Gender', 'Unit1', 'Unit2', 'HospAdmTime', 'ICULOS', 'SepsisLabel'], dtype='object')

- Vital signs (columns 1-8)
    - HR	Heart rate (beats per minute)
    - O2Sat	Pulse oximetry (%)
    - Temp	Temperature (Deg C)
    - SBP	Systolic BP (mm Hg)
    - MAP	Mean arterial pressure (mm Hg)
    - DBP	Diastolic BP (mm Hg)
    - Resp	Respiration rate (breaths per minute)
    - EtCO2	End tidal carbon dioxide (mm Hg)
- Laboratory values (columns 9-34)
    - BaseExcess	Measure of excess bicarbonate (mmol/L)
    - HCO3	Bicarbonate (mmol/L)
    - FiO2	Fraction of inspired oxygen (%)
    - pH	N/A
    - PaCO2	Partial pressure of carbon dioxide from arterial blood (mm Hg)
    - SaO2	Oxygen saturation from arterial blood (%)
    - AST	Aspartate transaminase (IU/L)
    - BUN	Blood urea nitrogen (mg/dL)
    - Alkalinephos	Alkaline phosphatase (IU/L)
    - Calcium	(mg/dL)
    - Chloride	(mmol/L)
    - Creatinine	(mg/dL)
    - Bilirubin_direct	Bilirubin direct (mg/dL)
    - Glucose	Serum glucose (mg/dL)
    - Lactate	Lactic acid (mg/dL)
    - Magnesium	(mmol/dL)
    - Phosphate	(mg/dL)
    - Potassium	(mmol/L)
    - Bilirubin_total	Total bilirubin (mg/dL)
    - TroponinI	Troponin I (ng/mL)
    - Hct	Hematocrit (%)
    - Hgb	Hemoglobin (g/dL)
    - PTT	partial thromboplastin time (seconds)
    - WBC	Leukocyte count (count*10^3/µL)
    - Fibrinogen	(mg/dL)
    - Platelets	(count*10^3/µL)
- Demographics (columns 35-40)
    - Age	Years (100 for patients 90 or above)
    - Gender	Female (0) or Male (1)
    - Unit1	Administrative identifier for ICU unit (MICU)
    - Unit2	Administrative identifier for ICU unit (SICU)
    - HospAdmTime	Hours between hospital admit and ICU admit
    - ICULOS	ICU length-of-stay (hours since ICU admit)

In [3]:
# check missing values
# get columns with missing values over 80%

In [4]:
ROOT_DIR = '../../../datasets/training_setA/training/'
total_pats = []
max_row = 0
min_row = 60
for file in os.listdir(ROOT_DIR):
    pat_df = pd.read_csv(ROOT_DIR+file, sep='|')
    pat_dict = (pat_df.isnull().sum() / pat_df.shape[0]).to_dict()
    pat_dict['pat']= file
    total_pats.append(pat_dict)
    if max_row < pat_df.shape[1]:
        max_row = pat_df.shape[1]
    if min_row > pat_df.shape[1]:
        min_row = pat_df.shape[1]

In [5]:
max_row

41

In [6]:
min_row

41

In [7]:
total_df = pd.DataFrame(total_pats)
total_df.describe()

,AST,Age,Alkalinephos,BUN,BaseExcess,Bilirubin_direct,Bilirubin_total,Calcium,Chloride,Creatinine,DBP,EtCO2,FiO2,Fibrinogen,Gender,Glucose,HCO3,HR,Hct,Hgb,HospAdmTime,ICULOS,Lactate,MAP,Magnesium,O2Sat,PTT,PaCO2,Phosphate,Platelets,Potassium,Resp,SBP,SaO2,SepsisLabel,Temp,TroponinI,Unit1,Unit2,WBC,pH
count,20336.000000,20336.0,20336.000000,20336.000000,20336.000000,20336.000000,20336.000000,20336.000000,20336.000000,20336.000000,20336.000000,20336.0,20336.000000,20336.000000,20336.0,20336.000000,20336.000000,20336.000000,20336.000000,20336.000000,20336.000000,20336.0,20336.000000,20336.000000,20336.000000,20336.000000,20336.000000,20336.000000,20336.000000,20336.000000,20336.000000,20336.000000,20336.000000,20336.000000,20336.0,20336.000000,20336.000000,20336.000000,20336.000000,20336.000000,20336.000000
mean,0.985239,0.0,0.985668,0.917105,0.897089,0.998583,0.987899,0.951119,0.915360,0.932715,0.505426,1.0,0.868923,0.992640,0.0,0.874364,0.918345,0.082537,0.879188,0.909895,0.000049,0.0,0.966439,0.108744,0.922877,0.127073,0.951533,0.913793,0.950586,0.933546,0.890158,0.103864,0.154459,0.949375,0.0,0.660033,0.998755,0.468234,0.468234,0.923271,0.885928
std,0.032535,0.0,0.032399,0.052101,0.127139,0.011021,0.025349,0.038369,0.057990,0.034599,0.424630,0.0,0.145305,0.028868,0.0,0.116203,0.052263,0.102391,0.084110,0.062169,0.007012,0.0,0.069675,0.125947,0.053689,0.154388,0.044905,0.104824,0.038400,0.037424,0.075777,0.139911,0.195842,0.093047,0.0,0.223556,0.009394,0.499002,0.499002,0.050486,0.135067
min,0.600000,0.0,0.600000,0.375000,0.000000,0.744681,0.642857,0.550000,0.375000,0.550000,0.000000,1.0,0.000000,0.333333,0.0,0.041667,0.375000,0.000000,0.166667,0.250000,0.000000,0.0,0.083333,0.000000,0.375000,0.000000,0.500000,0.111111,0.550000,0.558824,0.250000,0.000000,0.000000,0.083333,0.0,0.000000,0.761905,0.000000,0.000000,0.333333,0.000000
25%,0.980000,0.0,0.980769,0.896552,0.833333,1.000000,0.981132,0.932203,0.894737,0.918367,0.075000,1.0,0.750000,1.000000,0.0,0.846154,0.897436,0.019231,0.840909,0.885714,0.000000,0.0,0.959184,0.023256,0.900000,0.025641,0.931818,0.857143,0.932203,0.920000,0.857143,0.021978,0.028571,0.932203,0.0,0.650000,1.000000,0.000000,0.000000,0.904762,0.808511
50%,1.000000,0.0,1.000000,0.930233,0.944444,1.000000,1.000000,0.953488,0.931034,0.941176,0.388889,1.0,0.914894,1.000000,0.0,0.914894,0.931818,0.048780,0.900000,0.927273,0.000000,0.0,1.000000,0.066667,0.933333,0.071429,0.958333,0.952381,0.953488,0.942857,0.913043,0.055556,0.084746,1.000000,0.0,0.742857,1.000000,0.000000,0.000000,0.937500,0.936170
75%,1.000000,0.0,1.000000,0.950000,1.000000,1.000000,1.000000,0.976190,0.951220,0.954545,1.000000,1.0,1.000000,1.000000,0.0,0.946969,0.951220,0.111111,0.941272,0.951220,0.000000,0.0,1.000000,0.150000,0.954545,0.166667,0.980769,1.000000,0.975610,0.956057,0.944444,0.130435,0.200000,1.000000,0.0,0.782609,1.000000,1.000000,1.000000,0.954545,1.000000
max,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.0,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [8]:
pat_df.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,EtCO2,BaseExcess,HCO3,FiO2,pH,PaCO2,SaO2,AST,BUN,Alkalinephos,Calcium,Chloride,Creatinine,Bilirubin_direct,Glucose,Lactate,Magnesium,Phosphate,Potassium,Bilirubin_total,TroponinI,Hct,Hgb,PTT,WBC,Fibrinogen,Platelets,Age,Gender,Unit1,Unit2,HospAdmTime,ICULOS,SepsisLabel
0,79.0,100.0,36.20,122.5,77.5,55.0,8.0,NaN,1.0,NaN,NaN,7.36,43.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,195.0,NaN,NaN,NaN,4.1,NaN,NaN,32.4,NaN,30.3,NaN,NaN,171.0,70.56,1,0,1,-3.8,2,0
1,76.5,100.0,36.15,118.5,75.0,54.5,19.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.56,1,0,1,-3.8,3,0
2,82.0,96.0,36.40,103.0,68.0,53.0,24.0,NaN,-6.0,NaN,0.45,7.33,35.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.56,1,0,1,-3.8,4,0
3,91.5,97.0,36.90,93.5,62.0,48.5,26.0,NaN,-7.0,NaN,NaN,7.32,35.0,97.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,178.0,NaN,NaN,NaN,NaN,NaN,NaN,37.0,12.2,NaN,NaN,NaN,NaN,70.56,1,0,1,-3.8,5,0
4,90.5,98.5,37.25,106.0,71.0,56.5,25.0,NaN,-6.0,NaN,NaN,7.34,33.0,98.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,70.56,1,0,1,-3.8,6,0


In [9]:
removal = (total_df.mean()>.9).to_dict()
drop_cols = list(map(lambda x: x[0], filter(lambda x: x[1]==True, removal.items())))
drop_cols

['AST',
 'Alkalinephos',
 'BUN',
 'Bilirubin_direct',
 'Bilirubin_total',
 'Calcium',
 'Chloride',
 'Creatinine',
 'EtCO2',
 'Fibrinogen',
 'HCO3',
 'Hgb',
 'Lactate',
 'Magnesium',
 'PTT',
 'PaCO2',
 'Phosphate',
 'Platelets',
 'SaO2',
 'TroponinI',
 'WBC']

In [10]:
drop_cols.extend(['Unit1', 'Unit2'])
pat_df.drop(columns=drop_cols, inplace=True)
pat_df.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,FiO2,pH,Glucose,Potassium,Hct,Age,Gender,HospAdmTime,ICULOS,SepsisLabel
0,79.0,100.0,36.20,122.5,77.5,55.0,8.0,1.0,NaN,7.36,195.0,4.1,32.4,70.56,1,-3.8,2,0
1,76.5,100.0,36.15,118.5,75.0,54.5,19.0,NaN,NaN,NaN,NaN,NaN,NaN,70.56,1,-3.8,3,0
2,82.0,96.0,36.40,103.0,68.0,53.0,24.0,-6.0,0.45,7.33,NaN,NaN,NaN,70.56,1,-3.8,4,0
3,91.5,97.0,36.90,93.5,62.0,48.5,26.0,-7.0,NaN,7.32,178.0,NaN,37.0,70.56,1,-3.8,5,0
4,90.5,98.5,37.25,106.0,71.0,56.5,25.0,-6.0,NaN,7.34,NaN,NaN,NaN,70.56,1,-3.8,6,0


In [11]:
ROOT_DIR = '../../../datasets/training_setA/training/'
total_pats_df = pd.DataFrame()
for file in os.listdir(ROOT_DIR):
    pat_df = pd.read_csv(ROOT_DIR+file, sep='|')
    pat_df.drop(columns=drop_cols, inplace=True)
    pat_df['patient']=file
    total_pats_df = pd.concat([total_pats_df, pat_df], ignore_index=True)   

In [12]:
total_pats_df.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,FiO2,pH,Glucose,Potassium,Hct,Age,Gender,HospAdmTime,ICULOS,SepsisLabel,patient
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,59.59,1,-1.8,1,0,p008854.psv
1,85.0,97.0,NaN,130.0,90.00,NaN,22.0,NaN,NaN,NaN,NaN,NaN,NaN,59.59,1,-1.8,2,0,p008854.psv
2,96.0,96.0,NaN,124.0,84.67,NaN,23.0,NaN,NaN,NaN,142.0,3.9,38.6,59.59,1,-1.8,3,0,p008854.psv
3,95.0,95.0,NaN,124.0,82.00,NaN,23.0,NaN,NaN,NaN,NaN,NaN,NaN,59.59,1,-1.8,4,0,p008854.psv
4,94.0,95.0,NaN,116.0,78.00,NaN,20.0,NaN,NaN,NaN,NaN,NaN,NaN,59.59,1,-1.8,5,0,p008854.psv


In [13]:
total_pats_df.shape

(790215, 19)

In [14]:
total_pats_df = total_pats_df.fillna(0)
total_pats_df.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,FiO2,pH,Glucose,Potassium,Hct,Age,Gender,HospAdmTime,ICULOS,SepsisLabel,patient
0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,59.59,1,-1.8,1,0,p008854.psv
1,85.0,97.0,0.0,130.0,90.00,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,59.59,1,-1.8,2,0,p008854.psv
2,96.0,96.0,0.0,124.0,84.67,0.0,23.0,0.0,0.0,0.0,142.0,3.9,38.6,59.59,1,-1.8,3,0,p008854.psv
3,95.0,95.0,0.0,124.0,82.00,0.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,59.59,1,-1.8,4,0,p008854.psv
4,94.0,95.0,0.0,116.0,78.00,0.0,20.0,0.0,0.0,0.0,0.0,0.0,0.0,59.59,1,-1.8,5,0,p008854.psv


In [96]:
train, test = train_test_split(total_pats_df, test_size=0.2)

In [97]:
train.shape

(632172, 19)

In [98]:
test.shape

(158043, 19)

In [20]:
# y = train['SepsisLabel']
# train.drop(columns='SepsisLabel', inplace=True)

/home/madigun/.pyenv/versions/3.6.7/envs/grace_jupyter/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [99]:
train.head()

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,FiO2,pH,Glucose,Potassium,Hct,Age,Gender,HospAdmTime,ICULOS,SepsisLabel,patient
68156,82.0,98.0,0.0,0.0,0.0,0.0,23.0,0.0,0.0,0.0,0.0,0.0,0.0,28.21,0,-5.47,23,0,p011242.psv
779182,88.0,95.0,37.8,108.0,74.0,54.0,16.0,0.0,0.0,0.0,0.0,0.0,0.0,59.96,1,-2.99,22,0,p000446.psv
266054,133.0,100.0,0.0,110.0,81.0,64.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,41.32,0,-80.21,19,0,p014058.psv
379960,74.0,98.0,0.0,100.0,71.0,52.0,15.0,0.0,0.0,0.0,0.0,0.0,0.0,49.45,0,-5.40,11,0,p014238.psv
784758,92.0,0.0,0.0,151.0,97.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,44.00,0,-0.05,15,0,p009528.psv


In [100]:
train.groupby('patient').HR.count().describe()

count    20336.000000
mean        31.086349
std         18.047355
min          3.000000
25%         20.000000
50%         31.000000
75%         38.000000
max        281.000000
Name: HR, dtype: float64

In [139]:
train_dataset = pd.merge(train, train.groupby('patient').ICULOS.nunique(), how='left', on='patient', suffixes=('', '_CNT'))

In [140]:
max_len = train_dataset.ICULOS_CNT.max()

In [161]:
default_val = pd.DataFrame(train_dataset.iloc[:, :-3].min()).transpose()
default_val.ICULOS = 0

In [162]:
pd.concat([default_val] * 5)

,HR,O2Sat,Temp,SBP,MAP,DBP,Resp,BaseExcess,FiO2,pH,Glucose,Potassium,Hct,Age,Gender,HospAdmTime,ICULOS
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-30.0,0.0,0.0,0.0,0.0,0.0,18.11,0.0,-3710.66,0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-30.0,0.0,0.0,0.0,0.0,0.0,18.11,0.0,-3710.66,0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-30.0,0.0,0.0,0.0,0.0,0.0,18.11,0.0,-3710.66,0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-30.0,0.0,0.0,0.0,0.0,0.0,18.11,0.0,-3710.66,0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-30.0,0.0,0.0,0.0,0.0,0.0,18.11,0.0,-3710.66,0


In [194]:
import numpy as np
import sys
import datetime

In [195]:
train_dataset.patient.nunique()

20336

In [196]:
train_X = []
cnt = -1
start_time = datetime.datetime.now()
for grp, data in train_dataset.groupby('patient'):
    train_X.append( pd.concat([default_val] * (max_len - data.ICULOS_CNT.max()) + [data.iloc[:, :-3]] ).values )
    cnt += 1
    sys.stdout.write('\r[%d/%d] ' % (cnt, train_dataset.patient.nunique()) + str(datetime.datetime.now() - start_time))    

[20335/20336] 0:35:35.597569

In [197]:
train_X = np.array(train_X)

In [206]:
y = train_dataset.groupby('patient').SepsisLabel.unique().apply(lambda x: max(x))

In [207]:
train_X.shape, len(y)

((20336, 281, 17), 20336)

In [124]:
train_dataset = pd.merge(train, train.groupby('patient').ICULOS.nunique(), how='left', on='patient', suffixes=('', '_CNT'))
# train_dataset = train_dataset[train_dataset.ICULOS_CNT >= 30]
# train_dataset = train_dataset.sort_values(['patient', 'ICULOS']).groupby('patient').tail(30)

#  (size, timestep, feature)
y = train_dataset.groupby('patient').SepsisLabel.unique().apply(lambda x: max(x))
train_dataset = train_dataset[train_dataset.columns.tolist()[:-3]].values.reshape(train_dataset.patient.nunique(), -1, len(train_dataset.columns)-3)

In [125]:
train_dataset.shape

(11056, 30, 17)

In [92]:
max_length

273

In [199]:
model = Sequential()
# (time_step, feature_n)
# model.add(LSTM(100, activation='relu', input_shape=(max_length, 17)))
# model.add(LSTM(100, activation='relu', input_shape=(30, 17)))
model.add(LSTM(100, activation='relu', input_shape=(max_len, 17)))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

In [200]:
# SVG(model_to_dot(model, show_shapes=True).create(prog='dot', format='svg'))

In [201]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 100)               47200     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 101       
Total params: 47,301
Trainable params: 47,301
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# model.fit(train_dataset, y, epochs=1000, verbose=0)
model.fit(train_X, y, epochs=1000, verbose=0)